## Let's test wheter we can access the FPGA module using DMA 

In [ ]:
import math#from pynq import Overlay
from PIL import Image as PIL_Image
from PIL import ImageDraw as PIL_ImageDraw
from PIL import ImageFont
import matplotlib.pyplot as plt
import numpy as np
import math
import cv2
from pynq import MMIO
# from pynq import Xlnk
import time
import ctypes

In [ ]:
## Load bitstream and overlay 

In [ ]:
from pynq import (allocate, Overlay)
import numpy as np
ol = Overlay('./hw12.bit')

In [ ]:
ol.ip_dict

#You should see  axi_dma_0 and example_0 in the ip dict along with other zynq libraries. 

In [ ]:
ol.axi_dma_0?


###  Creat DMA instances 
Using the labels for the DMAs listed above, we can create two DMA objects.

In [ ]:
dma = ol.axi_dma_0
dma_send = ol.axi_dma_0.sendchannel
dma_recv = ol.axi_dma_0.recvchannel

### Load your Covolution layer module 

In [ ]:
example_ip=ol.example_0

In [ ]:
help(example_ip)

set up convolution layer to be read (set up AP_CTRL signals using MMIO)
bit 1: AP_start 

In [ ]:
example=ol.example_0
example.write(0,1)  


In [ ]:
example.read(0)

In [ ]:
data_size = 100
input_buffer = allocate(shape=(data_size,), dtype=np.int32)

In [ ]:

for i in range(data_size):
    input_buffer[i] = i+800;

In [ ]:
for i in range(10):
    print(hex(input_buffer[i]))

create receiving array structure 

In [ ]:
output_buffer = allocate(shape=(data_size,), dtype=np.int32)

for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

Now send the data to DMA and conv module 

In [ ]:
start_time=time.time()
dma_send.transfer(input_buffer)

Now receive the data 

In [ ]:
dma_recv.transfer(output_buffer)

In [ ]:
status_value = example.read(0)
while(status_value!=4):
    time.sleep(1)
    status_value=example.read(0)

end_time = time.time()
exec_time = end_time - start_time 
print("Elasped time :{}".format(exec_time))


print out the first few byts 

In [ ]:
for i in range(10):
    print('0x' + format(output_buffer[i], '02x'))

Check the DMA register 

In [ ]:
dma.register_map

In [ ]:
print("Input buffer address   :", hex(input_buffer.physical_address))
print("Output buffer address  :", hex(output_buffer.physical_address))
print("---")
print("DMA Source address     :", hex(dma.register_map.MM2S_SA.Source_Address))
print("DMA Destination address:", hex(dma.register_map.S2MM_DA.Destination_Address))

check convolution module status // bit 0: AP_START bit 1: AP_DONE, bit 2: AP_IDLE  (6 means it's done and idle) 